In [21]:
import pandas as pd
from arcgis import GIS
gis = GIS()
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import requests
import arcpy

import geopandas as gpd

In [36]:
#strRelease = "WF TDM v9.0.1"
lstFieldsFromForecasts = ['SEGID','MF2023','MF2028','MF2032','MF2042','MF2050','ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','ADJHIST','DONOTUSE']

In [52]:
# Your initial DataFrame
dfSegmentFilenames = pd.DataFrame([
    ["F_AREA=='Iron' | F_AREA=='Cache' | F_AREA=='Dixie'" , 'data/segments/Segments_State_20231221_Draft.shp', True ],
    ["PLANAREA=='WFRC'", 'data/segments/WFv901_Segments_20240409_GeoKeys_AADTs_Factors.shp', False],
    ["PLANAREA=='MAG'", 'data/segments/WF_Segments_20240326_Draft.shp', False]
], columns=['filter', 'filename', 'updateRoute'])

# Initialize an empty list to store each filtered GeoDataFrame
_filteredGdfs = []

# Iterate through each row in your DataFrame
for _, row in dfSegmentFilenames.iterrows():
    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(row['filename'])
    
    # Use the `query` method to apply the filter from the 'filter' column
    _filteredGdf = gdf.query(row['filter'])
    
    if row['updateRoute']:
        # Assuming 'SEGID' is the correct column to apply this operation
        _filteredGdf.loc[:, 'ROUTE'] = _filteredGdf['SEGID'].apply(lambda x: x.split('_')[0] if '_' in x else x)

    # Add the filtered GeoDataFrame to the list
    _filteredGdfs.append(_filteredGdf)

# Concatenate all filtered GeoDataFrames into a single GeoDataFrame
gdfs = pd.concat(_filteredGdfs, ignore_index=True)

# Assuming you want to display or further process `gdfs`
display(gdfs)


C:\Users\bhereth\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,F2028,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0,0007_000.0,0.000,0.076,0.080800,Dixie,3032.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0007_000.1,0.076,0.297,0.225062,Dixie,3032.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0007_000.3,0.297,1.525,1.233974,Dixie,4357.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0007_001.5,1.525,2.399,0.875242,Dixie,4357.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0007_002.4,2.399,3.449,1.051900,Dixie,4357.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,0,UTA_9576,0.000,0.000,5.303283,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6691,0,UTA_9580,0.000,0.000,4.995125,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6692,0,UTA_9584,0.000,0.000,4.377226,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6693,0,UTA_9588,0.000,0.000,4.654122,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# add observed
dfObsVolSource = pd.read_csv('intermediate/obsvolsource.csv')
display(dfObsVolSource)
dfAadt = pd.read_csv('intermediate/aadt.csv')

dfGetObsAadt = pd.merge(dfObsVolSource,dfAadt,on='SOURCE')
display(dfGetObsAadt)

dfGetObsAadt = dfGetObsAadt[['SEGID','YEAR','AADT']]

dfGetObsAadt['YEAR'] = 'AADT' + dfGetObsAadt['YEAR'].astype(str)

dfGetObsAadt = dfGetObsAadt.drop_duplicates(subset=['SEGID','YEAR'])

dfGetObsAadt = dfGetObsAadt.pivot(index='SEGID',columns='YEAR', values='AADT').fillna(0).reset_index()

display(dfGetObsAadt)

,SOURCE,SUBAREAID
0,AADTHistory.xlsx,0
1,AADTHistory.xlsx,2
2,AADTHistory.xlsx,3
3,AADTHistory.xlsx,4
4,AADTHistory.xlsx,5
5,WF_Segments_20240326_Draft.shp,1


,SOURCE,SUBAREAID,SEGID,YEAR,AADT
0,AADTHistory.xlsx,0,0065_000.1,1981,530
1,AADTHistory.xlsx,0,0065_000.1,1982,545
2,AADTHistory.xlsx,0,0065_000.1,1983,495
3,AADTHistory.xlsx,0,0065_000.1,1984,555
4,AADTHistory.xlsx,0,0065_000.1,1985,575
...,...,...,...,...,...
702380,WF_Segments_20240326_Draft.shp,1,MAG_6604,2018,500
702381,WF_Segments_20240326_Draft.shp,1,MAG_6604,2019,500
702382,WF_Segments_20240326_Draft.shp,1,MAG_6604,2020,500
702383,WF_Segments_20240326_Draft.shp,1,MAG_6604,2021,500


YEAR,SEGID,AADT1981,AADT1982,AADT1983,AADT1984,AADT1985,AADT1986,AADT1987,AADT1988,AADT1989,...,AADT2013,AADT2014,AADT2015,AADT2016,AADT2017,AADT2018,AADT2019,AADT2020,AADT2021,AADT2022
0,0006_000.0,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0
1,0006_000.7,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0
2,0006_016.0,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0
3,0006_046.0,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,340.0,360.0,390.0,412.0,366.0,369.0,372.0,385.0,424.0,395.0
4,0006_060.2,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,340.0,360.0,390.0,412.0,366.0,369.0,372.0,385.0,424.0,395.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7281,WFRC_8010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1000.0,1053.0,1081.0,1092.0,1112.0,993.0,1076.0,1087.0
7282,WFRC_8220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7555.0,7635.0,15375.0,16191.0,16628.0,16794.0,17096.0,15267.0,16549.0,16715.0
7283,WFRC_8262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13055.0,13200.0,13770.0,14498.0,6489.0,6554.0,6672.0,5958.0,6458.0,6523.0
7284,WFRC_8419,15075.0,15150.0,15150.0,15440.0,15850.0,16010.0,16170.0,16250.0,15430.0,...,15630.0,15800.0,16480.0,17353.0,17822.0,18000.0,18324.0,16363.0,17737.0,17915.0


In [39]:
# Assuming gdfs and dfGetObsAadt are already defined and loaded
# Step 1: Merge the dataframes on 'SEGID'
merged_df = pd.merge(gdfs, dfGetObsAadt, on='SEGID', suffixes=('', '_new'))

# Step 2: Update the columns in gdfs with those from dfGetObsAadt where they exist
# Perform an inner merge to get only the rows with matching SEGIDs
common_columns = [col for col in dfGetObsAadt.columns if col in gdfs.columns and col != 'SEGID']
merged_common = gdfs.merge(merged_df, on='SEGID', how='inner')

# Update only the matching rows in gdfs
for col in common_columns:
    gdfs.loc[gdfs['SEGID'].isin(merged_common['SEGID']), col] = merged_common[col + '_new'].values

display(gdfs)

,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,F2028,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0,0007_000.0,0.000,0.076,0.080800,Dixie,3032.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0007_000.1,0.076,0.297,0.225062,Dixie,3032.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0007_000.3,0.297,1.525,1.233974,Dixie,4357.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0007_001.5,1.525,2.399,0.875242,Dixie,4357.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0007_002.4,2.399,3.449,1.051900,Dixie,4357.0,3.0,53.0,Dixie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,0,UTA_9576,0.000,0.000,5.303283,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6691,0,UTA_9580,0.000,0.000,4.995125,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6692,0,UTA_9584,0.000,0.000,4.377226,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6693,0,UTA_9588,0.000,0.000,4.654122,MAG,0.0,1.0,49.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# double check that the above merge worked by looking at an I15 segment
gdfs[gdfs['SEGID']=='0015_295.6'][['SEGID','AADT2017']]

,SEGID,AADT2017
1371,0015_295.6,261473.0


In [41]:
# drop unneeded columns and fill nas
sdfSegments = GeoAccessor.from_geodataframe(gdfs)
sdfSegments.drop(columns=['Id','F_AREA'], inplace=True) # only used for art of forecasting tool
sdfSegments

,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,CCSGROUP19,FACMANADJ,...,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE
0,0007_000.0,0.000,0.076,0.080800,Dixie,3032.0,3.0,53.0,COW,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[268173.08999999985, 4101180.67], ..."
1,0007_000.1,0.076,0.297,0.225062,Dixie,3032.0,3.0,53.0,COX,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[268284.90490000043, 4101114.28910..."
2,0007_000.3,0.297,1.525,1.233974,Dixie,4357.0,3.0,53.0,COX,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[268529.2000000002, 4100859.150000..."
3,0007_001.5,1.525,2.399,0.875242,Dixie,4357.0,3.0,53.0,COX,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[269520.8300000001, 4099147.59], [..."
4,0007_002.4,2.399,3.449,1.051900,Dixie,4357.0,3.0,53.0,COX,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[270539.79000000004, 4098210.0], [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,UTA_9576,0.000,0.000,5.303283,MAG,0.0,1.0,49.0,TRN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{""paths"": [[[438326.5963000003, 4459103.093800..."
6691,UTA_9580,0.000,0.000,4.995125,MAG,0.0,1.0,49.0,TRN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{""paths"": [[[445692.733, 4445756.658399999], [..."
6692,UTA_9584,0.000,0.000,4.377226,MAG,0.0,1.0,49.0,TRN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{""paths"": [[[442158.45150000043, 4440093.24929..."
6693,UTA_9588,0.000,0.000,4.654122,MAG,0.0,1.0,49.0,TRN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{""paths"": [[[437417.13790000044, 4434294.84259..."


In [42]:
sdfSegments.groupby(['PLANAREA']).agg(COUNT=('SEGID','count'))

,COUNT
PLANAREA,
Cache,402
Dixie,555
MAG,2090
UDOT,396
WFRC,3252


In [43]:
# Connect to GIS
gis = GIS()

# Access the feature layer
feature_layer_url = 'https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Forecasts_gdb/FeatureServer/0'
feature_layer = FeatureLayer(feature_layer_url)

# Convert to Spatially Enabled DataFrame
sdfForecasts = feature_layer.query().sdf
sdfForecasts = sdfForecasts[lstFieldsFromForecasts] # filter columns by list

# Display the first few rows of the dataframe
display(sdfForecasts)


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST,DONOTUSE
0,0013_000.0,20000.0,21000.0,22000.0,21500.0,22000.0,0.0,0.0,-500.0,0.0,0.0,0.0,0
1,0013_000.6,20000.0,21000.0,21000.0,22000.0,23000.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0013_001.3,16000.0,17000.0,17000.0,18500.0,19500.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0013_001.6,17000.0,18000.0,18000.0,19500.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0013_001.8,17000.0,17500.0,17500.0,18500.0,19000.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,UDOT_7090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9632,UDOT_7091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9633,UDOT_7092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9634,UDOT_7093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [44]:
years = [2023, 2028, 2032, 2042, 2050]

for year in years:
    # F  : Forecast 
    # MF : Model Forecast (model volume + base year adjustment)
    # ADJ: Manual Adjustment
    sdfForecasts[f'F{year}'] = sdfForecasts[f'MF{year}'] + sdfForecasts[f'ADJ{year}'] + sdfForecasts['ADJHIST']

sdfForecasts

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST,DONOTUSE,F2023,F2028,F2032,F2042,F2050
0,0013_000.0,20000.0,21000.0,22000.0,21500.0,22000.0,0.0,0.0,-500.0,0.0,0.0,0.0,0,20000.0,21000.0,21500.0,21500.0,22000.0
1,0013_000.6,20000.0,21000.0,21000.0,22000.0,23000.0,0.0,0.0,0.0,0.0,0.0,0.0,0,20000.0,21000.0,21000.0,22000.0,23000.0
2,0013_001.3,16000.0,17000.0,17000.0,18500.0,19500.0,0.0,0.0,0.0,0.0,0.0,0.0,0,16000.0,17000.0,17000.0,18500.0,19500.0
3,0013_001.6,17000.0,18000.0,18000.0,19500.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17000.0,18000.0,18000.0,19500.0,20000.0
4,0013_001.8,17000.0,17500.0,17500.0,18500.0,19000.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17000.0,17500.0,17500.0,18500.0,19000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,UDOT_7090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
9632,UDOT_7091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
9633,UDOT_7092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
9634,UDOT_7093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [45]:
sdfForecasts.columns

Index(['SEGID', 'MF2023', 'MF2028', 'MF2032', 'MF2042', 'MF2050', 'ADJ2023',
       'ADJ2028', 'ADJ2032', 'ADJ2042', 'ADJ2050', 'ADJHIST', 'DONOTUSE',
       'F2023', 'F2028', 'F2032', 'F2042', 'F2050'],
      dtype='object')

In [46]:
display(sdfSegments.columns)

# Your initial list of fields you consider dropping, excluding 'SEGID'
columns_to_drop = [column for column in lstFieldsFromForecasts if column != 'SEGID']

# Calculate the intersection of columns to drop and existing columns
columns_to_drop_exist = [column for column in columns_to_drop if column in sdfSegments.columns]

# Drop these columns from sdfSegments if they exist
sdfSegments.drop(columns=columns_to_drop_exist, inplace=True)

# Display remaining columns in sdfSegments to verify
display(sdfSegments.columns)


Index(['SEGID', 'BMP', 'EMP', 'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID',
       'CO_FIPS', 'CCSGROUP19', 'FACMANADJ',
       ...
       'F2032', 'F2042', 'F2050', 'CH19TO50', 'CH19TO23', 'CH23TO28',
       'CH28TO32', 'CH32TO42', 'CH42TO50', 'SHAPE'],
      dtype='object', length=122)

Index(['SEGID', 'BMP', 'EMP', 'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID',
       'CO_FIPS', 'CCSGROUP19', 'FACMANADJ',
       ...
       'F2032', 'F2042', 'F2050', 'CH19TO50', 'CH19TO23', 'CH23TO28',
       'CH28TO32', 'CH32TO42', 'CH42TO50', 'SHAPE'],
      dtype='object', length=122)

In [54]:
sdfForecastsPlusSegments = pd.merge(sdfForecasts,sdfSegments.drop(columns=['F2019','F2023','F2028','F2032','F2042','F2050']),on='SEGID',how='right')
sdfForecastsPlusSegments

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,CUTRK2011,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE
0,0007_000.0,5800.0,5700.0,5900.0,9100.0,11000.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(268173.08999999985, 4101180.67), ..."
1,0007_000.1,3400.0,6800.0,16000.0,28500.0,37500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(268284.90490000043, 4101114.28910..."
2,0007_000.3,5500.0,13500.0,22500.0,42000.0,53000.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(268529.2000000002, 4100859.150000..."
3,0007_001.5,4800.0,7300.0,9700.0,23000.0,28500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(269520.8300000001, 4099147.59), (..."
4,0007_002.4,4800.0,7300.0,9100.0,13500.0,16500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(270539.79000000004, 4098210.0), (..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,UTA_9576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(438326.5963000003, 4459103.093800..."
6691,UTA_9580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(445692.733, 4445756.658399999), (..."
6692,UTA_9584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(442158.45150000043, 4440093.24929..."
6693,UTA_9588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(437417.13790000044, 4434294.84259..."


In [55]:
# cacluate and adjusted base year
sdfForecastsPlusSegments['F2019'] = sdfForecastsPlusSegments['AADT2019'] + sdfForecastsPlusSegments['ADJHIST']

In [56]:
# calculate change fields
periods = [
    ('19TO50', 'F2019', 'F2050'),
    ('19TO23', 'F2019', 'F2023'),
    ('23TO28', 'F2023', 'F2028'),
    ('28TO32', 'F2028', 'F2032'),
    ('32TO42', 'F2032', 'F2042'),
    ('42TO50', 'F2042', 'F2050')
]

for period, start_year_key, end_year_key in periods:
    ch_key = f'CH{period}'
    sdfForecastsPlusSegments[ch_key] = sdfForecastsPlusSegments[end_year_key] - sdfForecastsPlusSegments[start_year_key]

sdfForecastsPlusSegments


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE,F2019
0,0007_000.0,5800.0,5700.0,5900.0,9100.0,11000.0,0.0,0.0,0.0,0.0,...,NaN,NaN,7968.0,2768.0,-100.0,200.0,3200.0,1900.0,"{'paths': [[(268173.08999999985, 4101180.67), ...",3032.0
1,0007_000.1,3400.0,6800.0,16000.0,28500.0,37500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34468.0,368.0,3400.0,9200.0,12500.0,9000.0,"{'paths': [[(268284.90490000043, 4101114.28910...",3032.0
2,0007_000.3,5500.0,13500.0,22500.0,42000.0,53000.0,0.0,0.0,0.0,0.0,...,NaN,NaN,48643.0,1143.0,8000.0,9000.0,19500.0,11000.0,"{'paths': [[(268529.2000000002, 4100859.150000...",4357.0
3,0007_001.5,4800.0,7300.0,9700.0,23000.0,28500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,24143.0,443.0,2500.0,2400.0,13300.0,5500.0,"{'paths': [[(269520.8300000001, 4099147.59), (...",4357.0
4,0007_002.4,4800.0,7300.0,9100.0,13500.0,16500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,12143.0,443.0,2500.0,1800.0,4400.0,3000.0,"{'paths': [[(270539.79000000004, 4098210.0), (...",4357.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,UTA_9576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(438326.5963000003, 4459103.093800...",0.0
6691,UTA_9580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(445692.733, 4445756.658399999), (...",0.0
6692,UTA_9584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(442158.45150000043, 4440093.24929...",0.0
6693,UTA_9588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(437417.13790000044, 4434294.84259...",0.0


In [57]:
# drop do no use
sdfForecastsPlusSegments = sdfForecastsPlusSegments[sdfForecastsPlusSegments['DONOTUSE']==0]
sdfForecastsPlusSegments

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE,F2019
0,0007_000.0,5800.0,5700.0,5900.0,9100.0,11000.0,0.0,0.0,0.0,0.0,...,NaN,NaN,7968.0,2768.0,-100.0,200.0,3200.0,1900.0,"{'paths': [[(268173.08999999985, 4101180.67), ...",3032.0
1,0007_000.1,3400.0,6800.0,16000.0,28500.0,37500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34468.0,368.0,3400.0,9200.0,12500.0,9000.0,"{'paths': [[(268284.90490000043, 4101114.28910...",3032.0
2,0007_000.3,5500.0,13500.0,22500.0,42000.0,53000.0,0.0,0.0,0.0,0.0,...,NaN,NaN,48643.0,1143.0,8000.0,9000.0,19500.0,11000.0,"{'paths': [[(268529.2000000002, 4100859.150000...",4357.0
3,0007_001.5,4800.0,7300.0,9700.0,23000.0,28500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,24143.0,443.0,2500.0,2400.0,13300.0,5500.0,"{'paths': [[(269520.8300000001, 4099147.59), (...",4357.0
4,0007_002.4,4800.0,7300.0,9100.0,13500.0,16500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,12143.0,443.0,2500.0,1800.0,4400.0,3000.0,"{'paths': [[(270539.79000000004, 4098210.0), (...",4357.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,UTA_9576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(438326.5963000003, 4459103.093800...",0.0
6691,UTA_9580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(445692.733, 4445756.658399999), (...",0.0
6692,UTA_9584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(442158.45150000043, 4440093.24929...",0.0
6693,UTA_9588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(437417.13790000044, 4434294.84259...",0.0


In [58]:
lstDropFields = [item for item in lstFieldsFromForecasts if item != 'SEGID']
sdfForecastsPlusSegments.drop(columns=lstDropFields, inplace=True)
sdfForecastsPlusSegments

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,SEGID,F2023,F2028,F2032,F2042,F2050,BMP,EMP,DISTANCE,PLANAREA,...,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE,F2019
0,0007_000.0,5800.0,5700.0,5900.0,9100.0,11000.0,0.000,0.076,0.080800,Dixie,...,NaN,NaN,7968.0,2768.0,-100.0,200.0,3200.0,1900.0,"{'paths': [[(268173.08999999985, 4101180.67), ...",3032.0
1,0007_000.1,3400.0,6800.0,16000.0,28500.0,37500.0,0.076,0.297,0.225062,Dixie,...,NaN,NaN,34468.0,368.0,3400.0,9200.0,12500.0,9000.0,"{'paths': [[(268284.90490000043, 4101114.28910...",3032.0
2,0007_000.3,5500.0,13500.0,22500.0,42000.0,53000.0,0.297,1.525,1.233974,Dixie,...,NaN,NaN,48643.0,1143.0,8000.0,9000.0,19500.0,11000.0,"{'paths': [[(268529.2000000002, 4100859.150000...",4357.0
3,0007_001.5,4800.0,7300.0,9700.0,23000.0,28500.0,1.525,2.399,0.875242,Dixie,...,NaN,NaN,24143.0,443.0,2500.0,2400.0,13300.0,5500.0,"{'paths': [[(269520.8300000001, 4099147.59), (...",4357.0
4,0007_002.4,4800.0,7300.0,9100.0,13500.0,16500.0,2.399,3.449,1.051900,Dixie,...,NaN,NaN,12143.0,443.0,2500.0,1800.0,4400.0,3000.0,"{'paths': [[(270539.79000000004, 4098210.0), (...",4357.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,UTA_9576,0.0,0.0,0.0,0.0,0.0,0.000,0.000,5.303283,MAG,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(438326.5963000003, 4459103.093800...",0.0
6691,UTA_9580,0.0,0.0,0.0,0.0,0.0,0.000,0.000,4.995125,MAG,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(445692.733, 4445756.658399999), (...",0.0
6692,UTA_9584,0.0,0.0,0.0,0.0,0.0,0.000,0.000,4.377226,MAG,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(442158.45150000043, 4440093.24929...",0.0
6693,UTA_9588,0.0,0.0,0.0,0.0,0.0,0.000,0.000,4.654122,MAG,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(437417.13790000044, 4434294.84259...",0.0


In [59]:
sdfForecastsPlusSegments.columns


# Set display options to ensure no truncation
pd.set_option('display.max_columns', None)  # Ensures all columns are shown
pd.set_option('display.max_seq_items', None)  # For potentially long sequences

# Now, when you print or display the DataFrame, all column names will be shown
print(sdfForecastsPlusSegments.columns)

# Reset display options to default
pd.reset_option('display.max_columns')
pd.reset_option('display.max_seq_items')


Index(['SEGID', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050', 'BMP', 'EMP',
       'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID', 'CO_FIPS',
       'CCSGROUP19', 'FACMANADJ', 'FAC_APR', 'FAC_AUG', 'FAC_DEC', 'FAC_FAL',
       'FAC_FEB', 'FAC_FRI', 'FAC_JAN', 'FAC_JUL', 'FAC_JUN', 'FAC_MAR',
       'FAC_MAX', 'FAC_MAXMO', 'FAC_MAY', 'FAC_MON', 'FAC_NOV', 'FAC_OCT',
       'FAC_SAT', 'FAC_SEP', 'FAC_SPR', 'FAC_SUM', 'FAC_SUN', 'FAC_THU',
       'FAC_TUE', 'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WED', 'FAC_WEMAX', 'FAC_WIN',
       'ROUTE', 'Id_x', 'Id_y', 'AADTSTN', 'AADT2022', 'AADT2021', 'AADT2020',
       'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013',
       'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007',
       'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001',
       'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995',
       'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989',
      

In [60]:
lstFinalCols = ['SEGID',
                'ROUTE', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS', 'SUBAREAID', 'PLANAREA',
                'F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050',
                'CH19TO50', 'CH19TO23', 'CH23TO28', 'CH28TO32', 'CH32TO42', 'CH42TO50', 
                'AADTSTN',
                'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017',
                'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011',
                'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005',
                'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999',
                'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993',
                'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987',
                'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981',
                'SUTRK2022', 'CUTRK2022', 'SUTRK2021', 'CUTRK2021', 'SUTRK2020',
                'CUTRK2020', 'SUTRK2019', 'CUTRK2019', 'SUTRK2018', 'CUTRK2018',
                'SUTRK2017', 'CUTRK2017', 'SUTRK2016', 'CUTRK2016', 'SUTRK2015',
                'CUTRK2015', 'SUTRK2014', 'CUTRK2014', 'SUTRK2013', 'CUTRK2013',
                'SUTRK2012', 'CUTRK2012', 'SUTRK2011', 'CUTRK2011', 'SUTRK2010',
                'CUTRK2010', 'CCSGROUP19',
                'FAC_MON', 'FAC_TUE', 'FAC_WED', 'FAC_THU', 'FAC_FRI', 'FAC_SAT', 'FAC_SUN',
                'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WEMAX',
                'FAC_SPR', 'FAC_SUM', 'FAC_FAL', 'FAC_WIN', 
                'FAC_JAN', 'FAC_FEB', 'FAC_MAR', 'FAC_APR', 'FAC_MAY', 'FAC_JUN',
                'FAC_JUL', 'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC',
                'FAC_MAXMO','FAC_MAX',
                'FACMANADJ', 'SHAPE']


sdfForecastsPlusSegmentsFinal = sdfForecastsPlusSegments[lstFinalCols].copy()

# For numeric columns
numeric_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['int', 'float']).columns
sdfForecastsPlusSegmentsFinal[numeric_cols] = sdfForecastsPlusSegmentsFinal[numeric_cols].fillna(0)

# For string/object columns
string_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['object']).columns
sdfForecastsPlusSegmentsFinal[string_cols] = sdfForecastsPlusSegmentsFinal[string_cols].fillna('')

# If you also have categorical columns and want to treat them as strings:
categorical_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['category']).columns
if not categorical_cols.empty:  # Check if there are any categorical columns
    sdfForecastsPlusSegmentsFinal[categorical_cols] = sdfForecastsPlusSegmentsFinal[categorical_cols].fillna('')

sdfForecastsPlusSegmentsFinal

,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,F2019,F2023,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0007_000.0,0007,0.000,0.076,0.080800,53.0,3.0,Dixie,3032.0,5800.0,...,1.0082,1.0330,1.0393,1.0366,0.9847,0.9221,4,1.0710,0.0,"{'paths': [[(268173.08999999985, 4101180.67), ..."
1,0007_000.1,0007,0.076,0.297,0.225062,53.0,3.0,Dixie,3032.0,3400.0,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10,1.1264,0.0,"{'paths': [[(268284.90490000043, 4101114.28910..."
2,0007_000.3,0007,0.297,1.525,1.233974,53.0,3.0,Dixie,4357.0,5500.0,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10,1.1264,0.0,"{'paths': [[(268529.2000000002, 4100859.150000..."
3,0007_001.5,0007,1.525,2.399,0.875242,53.0,3.0,Dixie,4357.0,4800.0,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10,1.1264,0.0,"{'paths': [[(269520.8300000001, 4099147.59), (..."
4,0007_002.4,0007,2.399,3.449,1.051900,53.0,3.0,Dixie,4357.0,4800.0,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10,1.1264,0.0,"{'paths': [[(270539.79000000004, 4098210.0), (..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,UTA_9576,UTA,0.000,0.000,5.303283,49.0,1.0,MAG,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0.0,"{'paths': [[(438326.5963000003, 4459103.093800..."
6691,UTA_9580,UTA,0.000,0.000,4.995125,49.0,1.0,MAG,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0.0,"{'paths': [[(445692.733, 4445756.658399999), (..."
6692,UTA_9584,UTA,0.000,0.000,4.377226,49.0,1.0,MAG,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0.0,"{'paths': [[(442158.45150000043, 4440093.24929..."
6693,UTA_9588,UTA,0.000,0.000,4.654122,49.0,1.0,MAG,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0.0,"{'paths': [[(437417.13790000044, 4434294.84259..."


In [61]:
sdfForecastsPlusSegmentsFinal.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\Traffic-Volume-Historic-and-Forecast.shp'